In [1]:
import glob
import cv2
import skimage.io as io
import numpy as np
import pandas as pd
import pydicom
import pydicom.data
from PIL import Image
from pydicom.pixel_data_handlers.util import apply_voi_lut
from sklearn.model_selection import train_test_split
import os
import random

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
import glob
files= glob.glob("C:\\Users\\user\\Documents\\DATASET_PNG\\**\\*.png",recursive = True)
files_inmeyok = glob.glob("C:\\Users\\user\\Documents\\DATASET_PNG\\INMEYOK\\**\\*.png",recursive = True)
files_iskemi = glob.glob("C:\\Users\\user\\Documents\\DATASET_PNG\\ISKEMI\\**\\*.png",recursive = True)
files_kanama = glob.glob("C:\\Users\\user\\Documents\\DATASET_PNG\\KANAMA\\**\\*.png",recursive = True)

In [3]:
#ORİJİNAL FOTOĞRAFLAR
img_list = []
labels = []
for file in files_inmeyok:
    img = io.imread(file, plugin = "simpleitk")
    labels.append(0)
    img_list.append(img)

for file in files_iskemi:
    img = io.imread(file, plugin = "simpleitk")
    labels.append(1)
    img_list.append(img)

for file in files_kanama:
    img = io.imread(file, plugin = "simpleitk")
    labels.append(2)
    img_list.append(img)


In [4]:
len(img_list), len(labels)

(6648, 6648)

In [6]:
labels[4420:4450]

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [30]:
train = np.array(img_list)
test = np.array(labels)

<ipython-input-30-3dfdab5e7590>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(img_list)


In [31]:

# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(train, test,
    test_size=0.2, shuffle = True, random_state = 42)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 42) # 0.25 x 0.8 = 0.2

In [35]:
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(y_train.shape))
print("y val shape: {}".format(y_test.shape))

X_train shape: (3988,)
X_test shape: (1330,)
y_train shape: (3988,)
y_test shape: (1330,)
X_val shape: (3988,)
y val shape: (1330,)


In [32]:
X_train.keys()

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [33]:
X_train.head(), X_test.head(), X_val.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [12]:
y_train.head(), y_test.head(), y_val.head()

(      0
 310   0
 4144  0
 2209  0
 802   0
 792   0,
       0
 2986  0
 3628  0
 5231  1
 4617  1
 6258  2,
       0
 3851  0
 6065  2
 3415  0
 3626  0
 438   0)

In [42]:
X_test.head()

,0
5278,"[[[0, 0, 0, 255], [0, 0, 0, 255], [0, 0, 0, 25..."
6149,"[[[0, 0, 0, 255], [0, 0, 0, 255], [0, 0, 0, 25..."
139,"[[[0, 0, 0, 255], [0, 0, 0, 255], [0, 0, 0, 25..."
3967,"[[[0, 0, 0, 255], [0, 0, 0, 255], [0, 0, 0, 25..."
6224,"[[[0, 0, 0, 255], [0, 0, 0, 255], [0, 0, 0, 25..."


In [43]:
y_test.head()

,0
5278,1
6149,2
139,0
3967,0
6224,2


In [21]:
train_path="C:\\Users\\user\\Desktop\\DATASET_NEW\\train"
test_path="C:\\Users\\user\\Desktop\\DATASET_NEW\\test\\TEST1"
val_path="C:\\Users\\user\\Desktop\\DATASET_NEW\\val\\VAL1"

In [39]:
def savetofolder(new_path,files_len,arrays):
    for n in range(0,int(files_len)):
        cv2.imwrite(os.path.join(new_path,"img{}.png".format(n)), arrays[n])
        if n == (files_len):
             print('{} image save'.format(n))

In [41]:
savetofolder(test_path,len(X_test),X_test)

In [48]:
savetofolder(val_path,len(y_test),X_val)

In [40]:
savetofolder(train_path,len(X_train),X_train)

TypeError: Expected Ptr<cv::UMat> for argument 'img'

In [4]:
train_files = glob.glob(train_path+"\\**\\*.png",recursive = True)
test_files = glob.glob(test_path+"\\**\\*.png",recursive = True)
val_files = glob.glob(val_path+"\\**\\*.png",recursive = True)
len(train_files), len(test_files), len(val_files)


(3987, 1330, 1330)

In [ ]:
len(train_files)

In [10]:
img_list2 = []
def our_preprocessing_function(files):
   # for file in files:
        img = io.imread(files, plugin = "simpleitk")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
       
        lab = cv2.cvtColor(img,cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit =5.0, tileGridSize = ((4,4)))
        cl = clahe.apply(l)
        cl =  cv2 . bilateralFilter ( cl, 3 , 80 , 80 )
        gray = cv2.cvtColor(cl, cv2.COLOR_BGR2RGB) 
        img = cv2.resize(img,(224,224))
        return gray
     #   img_list2.append(gray)
    #return img_list2

In [22]:
image_list = []
def our_preprocessing_function(files):
    for file in files:
        img = io.imread(file, plugin = "simpleitk")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(224,224))
        img_transf = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2HSV) 
        img_transf[:,:,2] = cv2.equalizeHist(img_transf[:,:,2]) 
        img4 = cv2.cvtColor(img_transf, cv2.COLOR_HSV2BGR)
        img4 =  cv2 . bilateralFilter ( img4, 3 , 80 , 80 )
        gray = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
        image_list.append(gray)
    

In [20]:
len(image_list)

3987

In [23]:
our_preprocessing_function(train_files)

In [17]:
def savetofolder2(new_path,files_len):
    for n in range(0,int(files_len)):
        cv2.imwrite(os.path.join(new_path,"img{}.png".format(n)), image_list[n])
        if n == (files_len):
             print('{} image save'.format(n))

In [24]:
prepro_path = train_path+ "\\PRE_PROCESSING\\"
savetofolder2(prepro_path,len(image_list))

In [5]:
import SimpleITK as sitk

In [6]:
for file in test_path:
    test = sitk.ReadImage(file, sitk.sitkUInt8)
len(test)

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk\Code\IO\src\sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "C" does not exist.

In [27]:
from keras.applications import inception_resnet_v2

img_rows, img_cols = 224,224

model = inception_resnet_v2.InceptionResNetV2(weights = "imagenet",
                   include_top = False,
                   input_shape =(img_rows,img_cols, 3))

In [28]:
for layer in model.layers:
    layer.trainable = False
    
model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

In [29]:
def layer_adder(bottom_model,num_classes):
    top_model=bottom_model.output
    top_model=GlobalAveragePooling2D()(top_model)
    top_model=Dense(1024,activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model=Dense(num_classes,activation='softmax')(top_model)
    return top_model
    #5epochIR katmanları bu çalışcak

In [30]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes=3#inmeyok,iskemi,kanama
FC_Head=layer_adder(model,num_classes)
model=Model(inputs=model.input,outputs=FC_Head)
print(model.summary())#bu çalışcak

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [49]:
ornek = train_files[1031]

In [37]:
len(prepro_path)

55

In [42]:
from keras.preprocessing.image import ImageDataGenerator

path="C:\\Users\\user\\Desktop\\DATASET_NEW\\"
validation_data_dir= path+ 'val'
train_data_dir = path+ "train2"
test_data_dir=path+'test'

batch_size=32

train_datagen=ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator=train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical')

validation_datagen=ImageDataGenerator(rescale=1./255)
validation_generator=validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical')
    
test_datagen=ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_rows,img_cols),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 3987 images belonging to 1 classes.
Found 1330 images belonging to 1 classes.
Found 1330 images belonging to 1 classes.


In [44]:
from tensorflow.keras import optimizers
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint=ModelCheckpoint('strokeinception.h15',
                           monitor='val_loss',
                           mod="min",
                           save_best_only=True,
                           verbose=1)
earlystop=EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=3,
                        verbose=1,
                        restore_best_weights=True)
callbacks=[earlystop,checkpoint]

"""opt = Adam(learning_rate=0.00005)
model.compile(loss='categorical_crossentropy', optimizer= opt, metrics=['accuracy'])"""
sgd = optimizers.SGD(learning_rate = 0.01, momentum = 0.9,)
model.compile(optimizer = sgd, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
epochs=4
history=model.fit(
             train_generator,
             epochs=epochs,
             callbacks=callbacks,
             validation_data=validation_generator)

Epoch 1/4
  2/125 [..............................] - ETA: 9:29 - loss: 12557.1099 - accuracy: 0.2500       

KeyboardInterrupt: 

In [39]:
optimizer = ['Adam', 'SGD', 'Adam','SGD']
epochs = [1,1,1,1]
EPOCH_STEPS = int(5318/32)
history_all = {}#bu çalışcak

In [41]:
from keras.callbacks import ModelCheckpoint,EarlyStopping#bu çalışcak
for i in range(len(optimizer)):
    print('Usnig optimizer: ' + optimizer[i] + ', Epoch: ' + str(epochs[i]))

    checkpoint=ModelCheckpoint('strokeinception_resne1.h15',
                           monitor='val_loss',
                           mod="min",
                           save_best_only=True,
                           verbose=1)
    callbacks=[checkpoint]
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer[i], 
                  metrics=['accuracy'])

    history = model.fit(
            train_generator,
           # val_generator,
            steps_per_epoch=EPOCH_STEPS,
            epochs=epochs[i],
            callbacks = callbacks,
            validation_data=validation_generator,
            shuffle=True
            )
    if len(history_all) == 0:
            history_all = {key: [] for key in history.history}
    
    for key in history_all:
            history_all[key].extend(history.history[key])

Usnig optimizer: Adam, Epoch: 1
 21/166 [==>...........................] - ETA: 5:47 - loss: 17.6602 - accuracy: 0.3498

KeyboardInterrupt: 